In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2 # OpenCV 사용을 위해 import 합니다. pip install opencv-python
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [15]:
# 하이퍼 파라미터 설정
IMG_WIDTH = 48  # VGG16은 최소 32x32 입력이 필요하다. MNIST(28x28)보다 크게 설정한다..
IMG_HEIGHT = 48
BATCH_SIZE = 128
EPOCHS = 10 # 파인튜닝이므로 많은 에포크가 필요하지 않을 수 있다.
LEARNING_RATE = 1e-5 # 파인튜닝 시에는 낮은 학습률을 사용한다.
NUM_CLASSES = 10

In [4]:
# MNIST 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# 데이터 차원 확장 (채널 추가) 및 크기 조정, 3채널 변환
def preprocess_data(images, labels, target_size=(IMG_WIDTH, IMG_HEIGHT)):
    """
    MNIST 데이터를 VGG16 입력 형식에 맞게 전처리하는 함수.
    - 이미지 크기 조정 (Resize)
    - 흑백 이미지를 3채널 (RGB) 이미지로 변환
    - 정규화 (Normalization)
    - 레이블 원-핫 인코딩
    """
    processed_images = []
    for img in images:
        # 크기 조정 (cv2 사용)
        img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
        # 3채널로 변환 (회색조 이미지를 3번 복사)
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
        processed_images.append(img_rgb)

    processed_images = np.array(processed_images, dtype='float32')
    # 픽셀 값 정규화 [0, 255] -> [0, 1]
    processed_images /= 255.0

    # 레이블 원-핫 인코딩
    processed_labels = to_categorical(labels, NUM_CLASSES)

    return processed_images, processed_labels

print("데이터 전처리를 시작합니다...")
x_train_processed, y_train_processed = preprocess_data(x_train, y_train)
x_test_processed, y_test_processed = preprocess_data(x_test, y_test)
print("데이터 전처리 완료.")
print("훈련 데이터 형태:", x_train_processed.shape) # (60000, 48, 48, 3)
print("테스트 데이터 형태:", x_test_processed.shape) # (10000, 48, 48, 3)
print("훈련 레이블 형태:", y_train_processed.shape) # (60000, 10)
print("테스트 레이블 형태:", y_test_processed.shape) # (10000, 10)

데이터 전처리를 시작합니다...
데이터 전처리 완료.
훈련 데이터 형태: (60000, 48, 48, 3)
테스트 데이터 형태: (10000, 48, 48, 3)
훈련 레이블 형태: (60000, 10)
테스트 레이블 형태: (10000, 10)


In [6]:
# VGG16 모델 로드 (사전 학습된 가중치 사용, 최상위 분류 레이어 제외)
input_tensor = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# VGG16 모델 로드
# include_top=False: ImageNet 분류기 부분을 제외하고 특징 추출 부분만 가져옴
# weights='imagenet': ImageNet으로 사전 학습된 가중치를 사용
# input_tensor: 위에서 정의한 입력 텐서를 사용
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [7]:
# 파인튜닝 전략 설정: 마지막 합성곱 블록만 학습 가능하도록 설정
# VGG16의 구조를 확인하고 어떤 레이어까지 동결할지 결정할 수 있다.
print(base_model.summary()) # 모델 구조 확인용

# 초기에는 모든 VGG16 레이어를 동결
for layer in base_model.layers:
    layer.trainable = False

# 마지막 합성곱 블록 ('block5_conv1', 'block5_conv2', 'block5_conv3')만 학습 허용
# 레이어 이름을 기반으로 설정
layers_to_unfreeze = ['block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']
print("\n다음 레이어들의 동결을 해제합니다 (파인튜닝 대상):")
for layer in base_model.layers:
    if layer.name in layers_to_unfreeze:
        layer.trainable = True
        print(f"- {layer.name}")
    else:
        layer.trainable = False # 혹시 모르니 다시 한번 동결 확인

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 48, 48, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 48, 48, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 48, 48, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None

다음 레이어들의 동결을 해제합니다 (파인튜닝 대상):
- block5_conv1
- block5_conv2
- block5_conv3
- block5_pool


In [10]:
# 새로운 분류기 레이어 추가
# VGG16의 특징 추출 부분 위에 새로운 분류기를 쌓는다.
x = base_model.output
x = Flatten(name='flatten')(x)
x = Dense(512, activation='relu', name='fc1')(x) # 은닉층 추가
x = Dropout(0.5, name='dropout1')(x) # 과적합 방지를 위한 드롭아웃
predictions = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x) # 출력층 (10개 클래스)

# 최종 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# 모델 컴파일
# 파인튜닝 시에는 매우 작은 학습률을 사용하는 것이 일반적이다.
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n모델 구성 완료. 모델 요약:")
model.summary()


모델 구성 완료. 모델 요약:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 48, 48, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 48, 48, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 48, 48, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 14,982,474 (57.15 MB)

 Trainable params: 7,347,210 (28.03 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [16]:
# 모델 학습 (파인튜닝)
print("\n모델 학습(파인튜닝)을 시작합니다...")
history = model.fit(x_train_processed, y_train_processed,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test_processed, y_test_processed),
                    verbose=1) # 학습 진행 상황 출력


모델 학습(파인튜닝)을 시작합니다...
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 613s 1s/step - accuracy: 0.8735 - loss: 0.4641 - val_accuracy: 0.9820 - val_loss: 0.0557
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 617s 1s/step - accuracy: 0.9773 - loss: 0.0762 - val_accuracy: 0.9872 - val_loss: 0.0377
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 648s 1s/step - accuracy: 0.9854 - loss: 0.0487 - val_accuracy: 0.9902 - val_loss: 0.0292
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 648s 1s/step - accuracy: 0.9888 - loss: 0.0388 - val_accuracy: 0.9912 - val_loss: 0.0259
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 648s 1s/step - accuracy: 0.9899 - loss: 0.0331 - val_accuracy: 0.9913 - val_loss: 0.0263
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 649s 1s/step - accuracy: 0.9908 - loss: 0.0291 - val_accuracy: 0.9922 - val_loss: 0.0218
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 649s 1s/step - accuracy: 0.9928 - loss: 0.0238 - val_accuracy: 0.9932 - val_loss: 0.0211
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 784s 2s/step - accuracy: 0.9931 - 

In [19]:
# 8. 모델 평가 (혼동 행렬 및 F1 스코어 포함)
print("\n모델 평가를 시작합니다...")
loss, accuracy = model.evaluate(x_test_processed, y_test_processed, verbose=0)
print(f"테스트 데이터 손실 (Loss): {loss:.4f}")
print(f"테스트 데이터 정확도 (Accuracy): {accuracy:.4f}")

# 예측 수행
y_pred_probs = model.predict(x_test_processed)
y_pred_labels = np.argmax(y_pred_probs, axis=1) # 확률에서 레이블로 변환

# 실제 레이블 (원-핫 인코딩 전의 y_test 사용)
y_true_labels = y_test # mnist.load_data()에서 로드한 원본 y_test 사용

# F1 스코어 계산 (macro average)
f1 = f1_score(y_true_labels, y_pred_labels, average='macro')
print(f"F1 스코어 (Macro Average): {f1:.4f}")

# 분류 리포트 출력 (정밀도, 재현율, F1 스코어 등)
print("\n분류 리포트:")
print(classification_report(y_true_labels, y_pred_labels, target_names=[str(i) for i in range(NUM_CLASSES)]))

# 혼동 행렬 계산
cm = confusion_matrix(y_true_labels, y_pred_labels)
print("\nconfusion matrix")
print(cm)


모델 평가를 시작합니다...
테스트 데이터 손실 (Loss): 0.0217
테스트 데이터 정확도 (Accuracy): 0.9923
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 130ms/step
F1 스코어 (Macro Average): 0.9923

분류 리포트:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       1.00      0.98      0.99       974
           9       0.98      1.00      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000


confusion matrix
[[ 977    0    1    0    0    0    1    0    0    1]
 [   0 1131 